In [10]:
import tensorflow
import keras
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Dropout,BatchNormalization,Dense,Flatten
from tensorflow.keras import layers
import numpy as np
import pandas as pd

In [5]:
image_size=(224,224)
batch_size=64
seed=1337

train_data_set=tensorflow.keras.preprocessing.image_dataset_from_directory(
    directory="data/traindata/train/",
    validation_split=0.20,
    subset="training",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
    interpolation='bilinear'
)
validation_data_set=tensorflow.keras.preprocessing.image_dataset_from_directory(
    directory="data/traindata/train/",
    validation_split=0.20,
    subset="validation",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
    interpolation='bilinear'
)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.


In [ ]:
import matplotlib.pylab as plt

plt.figure(figsize=(20,20))
for images,labels in train_data_set.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(int(labels[i]))
        plt.axis("off")

In [ ]:
# data augmentation in image for generating more data

import matplotlib.pylab as plt

data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
    ]
)

plt.figure(figsize=(20,20))
for images,_ in train_data_set.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(int(labels[i]))
        plt.axis("off")

In [14]:
def make_model():
    inputs=keras.Input(input_shape=(224,224))
    x = data_augmentation(inputs)
    x = layers.Rescaling(1./255)(x)

    # vgg-16 network

    model=Sequential()

    model.add(Conv2D(input_shape=(224,224,3),filters=64,kernel_size=(3,3),padding="same",activation="relu"))
    model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same",activation="relu"))
    model.add(MaxPooling2D(strides=(2,2),pool_size=(2,2)))

    model.add(Conv2D(filters=128,kernel_size=(3,3),padding="same",activation="relu"))
    model.add(Conv2D(filters=128,kernel_size=(3,3),padding="same",activation="relu"))
    model.add(MaxPooling2D(strides=(2,2),pool_size=(2,2)))

    model.add(Conv2D(filters=256,kernel_size=(3,3),padding="same",activation="relu"))
    model.add(Conv2D(filters=256,kernel_size=(3,3),padding="same",activation="relu"))
    model.add(Conv2D(filters=256,kernel_size=(3,3),padding="same",activation="relu"))
    model.add(MaxPooling2D(strides=(2,2),pool_size=(2,2)))

    model.add(Conv2D(filters=512,kernel_size=(3,3),padding="same",activation="relu"))
    model.add(Conv2D(filters=512,kernel_size=(3,3),padding="same",activation="relu"))
    model.add(Conv2D(filters=512,kernel_size=(3,3),padding="same",activation="relu"))
    model.add(MaxPooling2D(strides=(2,2),pool_size=(2,2)))

    model.add(Conv2D(filters=512,kernel_size=(3,3),padding="same",activation="relu"))
    model.add(Conv2D(filters=512,kernel_size=(3,3),padding="same",activation="relu"))
    model.add(Conv2D(filters=512,kernel_size=(3,3),padding="same",activation="relu"))
    model.add(MaxPooling2D(strides=(2,2),pool_size=(2,2)))

    model.add(Flatten())

    model.add(Dense(units=4096,activation="relu"))
    model.add(Dense(units=4096,activation="relu"))
    model.add(Dense(units=1000,activation="relu"))
    ## as out puts can be of two types
    model.add(Dense(units=2,activation="softmax"))
    model.summary()

    
    print(x)
make_model()
    

ValueError: Please provide to Input a `shape` or a `tensor` or a `type_spec` argument. Note that `shape` does not include the batch dimension.

In [ ]:
# now we have to classify the data manually.Means dogs and cats are kept in two sepertae
# folder.So that we can tell which belongs to which class.

import os,shutil
directory_path='data/traindata/'
sub_dir=os.listdir(directory_path)
# print(sub_dir)
file_path=os.path.join(directory_path,str(sub_dir[0]))

all_files=os.listdir(file_path)

cat_class=os.path.join(file_path,'cat')
dog_class=os.path.join(file_path,'dog')

cat_dir=os.mkdir(cat_class)
dog_dir=os.mkdir(dog_class)

for file in all_files:
    original=os.path.join(file_path,str(file))
    if str(file).split('.').count('cat')>0:
        target=os.path.join(file_path,'cat')
        shutil.move(original,target)
        print(str(file))
        #move to cat class 
    if str(file).split('.').count('dog')>0:
        target=os.path.join(file_path,'dog')
        shutil.move(original,target)
        print(str(file))
        #move to dog 
# print(all_files[0])

In [ ]:
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
# https://studymachinelearning.com/keras-imagedatagenerator-with-flow_from_directory/
train_data_gen=ImageDataGenerator(
        rotation_range=0,
        width_shift_range=0.5,
        height_shift_range=0.5,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest',
        validation_split=0.20)

test_data_gen=ImageDataGenerator(
        rescale=1./255
)
# categorical means return a 2d hot encoded image

# one_hot_encoding : One-Hot Encoding is another popular technique for treating categorical variables. It simply creates additional features based on the number of unique values in the categorical feature. Every unique value in the category will be added as a feature.

# https://vijayabhaskar96.medium.com/tutorial-image-classification-with-keras-flow-from-directory-and-generators-95f75ebe5720

batch_size=64
# small batch size leads to bigger BatchDataset
# shuffle: Whether to shuffle the data. Default: True. If set to False, sorts the data in alphanumeric order.
# seed: Optional random seed for shuffling and transformations.
seed=42
tr_data_dir_iterator=train_data_gen.flow_from_directory(
        directory="data/traindata/train/",
        batch_size=batch_size,
        class_mode='categorical',
        target_size=(224,224),
        seed=seed, 
        shuffle=True,
        subset='training')
valid_data_dir_iterator=train_data_gen.flow_from_directory(
        directory="data/traindata/train/",
        batch_size=batch_size,
        class_mode='categorical',
        target_size=(224,224), 
        seed=seed, 
        shuffle=True,
        subset='validation'
)


test_data_dir_iterator=test_data_gen.flow_from_directory(
        directory="data/testdata/",
        batch_size=1,
        class_mode=None,
        color_mode="rgb",
        target_size=(224,224),
        shuffle=False,
        seed=seed
)
print(tr_data_dir_iterator,valid_data_dir_iterator,test_data_dir_iterator)




In [ ]:
from tensorflow.keras.optimizers import Adam
opt = Adam(learning_rate=0.001)
model.compile(optimizer=opt, loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

In [ ]:
import os
import matplotlib as plt 
from matplotlib.pyplot import imread,imshow,get_cmap
train_dir="data/traindata/train/dog/"
dir_list=os.listdir(train_dir)

img=imread(os.path.join(train_dir,(str(dir_list[0]))))
imshow(img)